In [1]:
# install packages
!pip install pytesseract
!pip install pdf2image

You should consider upgrading via the 'C:\Users\lmy\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\lmy\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


In [2]:
# import packages
import pytesseract
from pdf2image import convert_from_path
import os
import re
from datetime import datetime
import pandas as pd

In [3]:
# Function to extract text from a PDF file
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'  

def extract_text_from_pdf(pdf_path):
    # Convert the PDF to a list of PIL Image objects
    images = convert_from_path(pdf_path)
    text_list = []
    # Loop through each page of the PDF and extract text using Tesseract
    for image in images:
        text = pytesseract.image_to_string(image)
        text_list.append(text)
    # Combine text from all pages into a single string
    text = '\n'.join(text_list)
    return text

In [285]:
# Define the regex pattern to match the date
footnote_pattern = r'^(\*|\d(?![./])|\d{1}/)\s.*'
title_pattern = r"1\.\s+\w+(\s+\w+)*"

In [11]:
date_pattern = r'\b\d{1,2}\s+(?:January|February|March|April|May|June|July|August|September|October|November|December)\s+\d{4}\b'
# extract year (A) and date (J)
def extract_date(text):
    date_match = re.search(date_pattern, text)
    if date_match:
        date = date_match.group(0)
        date_obj = datetime.strptime(date, '%d %B %Y')
        # change mm/dd/yyyy format
        new_date = date_obj.strftime('%m/%d/%Y') #column J
        year = new_date[-4:] #column A
    else:
        year = date = 'N/A'
        #print("No date found")
    return year, date

In [5]:
def remove_empty(text):
    text = text.replace('\n', ' ')
    text = text.strip()
    if text == '':
        return 'N/A'
    else:
        return text
    
    
council_pattern1 = r"(?<=session).*?(?=Agenda)"
council_pattern2 = r'([A-Z][A-Za-z]+\s)+(COUNCIL|Council)'

# extract council/committe name (B)
def extract_council(text):
    match1 = re.search(council_pattern1, text, re.DOTALL)
    match2 = re.search(council_pattern2, text, re.DOTALL)
    council = 'N/A'
    # handle the council after session
    if match1:
        council = match1.group(0).strip()
    # handle the council before session
    if match2:
        council = match2.group(0).strip()
    if 'Economic and Social' in text and 'COMMISSION ON HUMAN RIGHTS' in text:
        council = 'Economic and Social Council'
    return remove_empty(council)

In [6]:
session_pattern = r'(?P<session>[\w-]+\ssession)'
# extract session (C)
def extract_session(text):
    match = re.search(session_pattern, text)
    if match:
        session = match.group("session")
    else:
        session = 'N/A'
        print("Session information not found.")
    return session

In [7]:
agenda_pattern = r'Agenda item (\d+)(?: \((\w+)\))?(?:\n\n)?'
draft_pattern = r': *([\w ]*|[\n]*)? ?draft' #': *([\w ]*)?draft'

# extract Agenda item (D), Agenda detail (E), and countries (F)
def extract_agenda_countries(text):
    match1 = re.search(agenda_pattern, text)
    if match1:
        agenda_item_number = match1.group(1)
        agenda_item_letter = match1.group(2)
        if agenda_item_letter is not None:
            agenda_item = f"{agenda_item_number} ({agenda_item_letter})"
        else:
            agenda_item = agenda_item_number
        
        start_index = match1.end()

        match2 = re.search(draft_pattern, text[start_index:], re.DOTALL)
        if match2:
            end_index = start_index + match2.start()
            content = text[start_index:end_index]
            parts = content.split("\n\n")
            parts = [remove_empty(s) for s in parts]
            agenda_detail = parts[0]
            countries = ''
            for part in parts[1:]:
                if part[-2] == ':':
                    countries += part[:-2]
                else:
                    countries += part
        else:
            print('match2 error')
    # else:
    #     agenda_item = 'N/A'
    #     print("No match found")
    # if countries[-1] == ':':
    #     countries = countries[:-1]
    
    return agenda_item, remove_empty(agenda_detail), remove_empty(countries)

In [8]:
def split_text(text):
    # delete extracted text
    split_text = re.split(draft_pattern, text, flags=re.IGNORECASE)
    #print(split_text)
    #print(len(split_text))
    part1_text = split_text[0]
    part2_text = split_text[2]
    return part1_text, part2_text

In [22]:
text_head_pattern = r'(The General Assembly,|The Commission on Human Rights,|1\.)'
number_title_pattern = r'^\s*(\d{4})?/?\W*(.*)'
# extract body title number (G) and body title （H）
def extract_body_title(text):
    title_body_text = re.split(text_head_pattern, text)
    title_text = title_body_text[0]
    #print(title_text)
    title_text = title_text.replace('\n', ' ')
    
    match1 = re.search(r'[A-Z\d]', title_text)
    if match1:
        index = match1.start()
        title_text = title_text[index-1:]
        print(title_text) 
    else:
        print('No uppercase letter or digit found.')

    match2 = re.match(number_title_pattern, title_text)
    if match2:
        title_number = match2.group(1)
        if title_number == None:
            title_number = 'N/A'
        title = match2.group(2)
    else:
        title_number = 'N/A'
        title = 'N/A'

    return title_number, remove_empty(title)

In [300]:
# Function to classify text into column A to N
def classify_text_to_column(text):
    result = []

    part1_text, part2_text = split_text(text)
    # get year(A) and date(J)
    year, date = extract_date(part1_text)
    result.append(year)


    # get council/committe name (B)
    council = extract_council(part1_text)
    result.append(council)


    # get session (C)
    session = extract_session(part1_text)
    result.append(session)


    # get agenda item (D), agend detail (E), and countries (F)
    agenda_item, agenda_detail, countries = extract_agenda_countries(text)
    result.append(agenda_item)
    result.append(agenda_detail)
    result.append(countries)


    # get title number (G) and title body (H)
    title_number, title_text = extract_body_title(part2_text)
    result.append(title_number)
    result.append(title_text)


    # Body text (I)
    # extract the text content from the second non-empty paragraph
    body_text = ''
    footnote = ''
    footnote_idx = 1
    for paragraph in paragraphs[idx:]:
        #for line in paragraph.split('\n'):
            # exclude lines containing specific patterns
        paragraph = paragraph.replace("\n", " ")
        match = re.search(footnote_pattern, paragraph)
        if match:
            if match.group(0)[0] == '*':
                footnote += match.group(0)
            elif len(match.group(0)) < 90 and match.group(0)[0] == str(footnote_idx):
                footnote += match.group(0)
                footnote_idx += 1
        else:
            #if not re.search(r'^(\d{2}-\d{5}|[A-Za-z]/\d{2}/[A-Za-z]\.\d|[A-Za-z ]+Please recycle|Page \d{1})', paragraph):
            if len(paragraph) > 50 or paragraph == "The General Assembly,":
                body_text += paragraph.strip() + ' '
            else:
                if len(paragraph) == 0:
                    continue
                if paragraph[0] == str(footnote_idx) or paragraph[0] == "*":
                    footnote_idx += 1
                    footnote += paragraph 
    result.append(body_text)

    # date (J)
    result.append(date)
    
    # # file name (K) and country (L) will be obtained outside

    # footnote (M)
    result.append(footnote)

    # scanned (N), cannot know 
    if int(year) > 1990:
        scanned = 'yes'
    else:
        scanned = 'no'
    result.append(scanned)
    
    return result

In [107]:
# Path to the folder containing the PDF files
pdf_folder = './Dataset'

# Path to the folder where the CSV files will be saved
csv_folder = './csv_folder'

# csv file column list
column_list = ['year', 'Council', 'Session', 'Agenda item', 'Agenda detail', 'cosponsored countries', 
               'body title number',	'body title detail', 'body text', 'date', 'footnote', 'scanned', 
               'file', 'filecountry']

# Loop through each folder and PDF file and extract text
for country_folder in os.listdir(pdf_folder):
    print('---------------' + country_folder + '---------------' )
    if not os.path.isdir(os.path.join(pdf_folder, country_folder)):
        continue
    csv_path = os.path.join(csv_folder, f"{country_folder}.csv")
    rows = []
    count = 0
    # Loop through each file 
    for pdf_file in os.listdir(os.path.join(pdf_folder, country_folder)):
        if not pdf_file.endswith('.pdf'):
            continue
        # check if this file is old-version
        year = int(pdf_file.split('_')[0])
        if year < 1994:
            continue

        pdf_path = os.path.join(pdf_folder, country_folder, pdf_file).replace('\\', '/')
        print(pdf_path)
        # try:
        text = extract_text_from_pdf(pdf_path)
        text_columns = classify_text_to_column(text)
        row = text_columns + [pdf_file[:-4], country_folder] #add column K and L
        
        # check if year was extracted correctly
        if row[0] != year:
            row[0] = year
        
        rows.append(row)
        count += 1
        #print(rows)
        df = pd.DataFrame(rows, columns = column_list)
        df.to_csv(csv_path, index = False) 
        if count == 10:
            break
        # except:
        #     print(f"Error extracting text from {pdf_path}")
        #     continue
    break
    # Write the rows to the CSV file
    df = pd.DataFrame(rows, columns = column_list)
    df.to_csv(csv_path, index = False)  

---------------Afghanistan---------------
./Dataset/Afghanistan/1994_163336.pdf
./Dataset/Afghanistan/1994_163532.pdf
./Dataset/Afghanistan/1994_163619.pdf
./Dataset/Afghanistan/1994_163690.pdf
./Dataset/Afghanistan/1994_163780.pdf
./Dataset/Afghanistan/1994_163787.pdf
./Dataset/Afghanistan/1994_163866.pdf
./Dataset/Afghanistan/1994_163874.pdf
./Dataset/Afghanistan/1994_163890.pdf
./Dataset/Afghanistan/1994_163906.pdf


### TEST

In [23]:
test_list = ['1994_163532.pdf', '1994_163787.pdf', '1994_163866.pdf', '1999_395501.pdf', '2000_412394.pdf', '2006_584361.pdf'] 
for test_pdf in test_list:
    #if test_pdf == '1999_395501.pdf':
    test_text = extract_text_from_pdf(f'./Dataset/Afghanistan/{test_pdf}')
        #print(test_text)
        #break
    p1, p2 = split_text(test_text)
        #print(p1)
        #print(p2)
        # test year and date
    #print(extract_date(p1))
    #print(extract_council(p1))
    #print(extract_session(p1))
    #print(extract_agenda_countries(test_text))
    #remain = split_text(test_text)
    print(extract_body_title(p2))

No uppercase letter or digit found.
('N/A', 'resolution')
 A/49/L.8  
('N/A', 'A/49/L.8')
 Towards full integration of persons with disabilities in society:  implementation of the Standard Rules on the Equalization of  Opportunities for Persons with Disabilities, and of the long-term  strategy to implement the world programme of action concerning disabled persons to the year 2000 and beyond  
('N/A', 'Towards full integration of persons with disabilities in society:  implementation of the Standard Rules on the Equalization of  Opportunities for Persons with Disabilities, and of the long-term  strategy to implement the world programme of action concerning disabled persons to the year 2000 and beyond')
 Emergency international assistance for peace, normalcy and rehabilitation in Tajikistan  
('N/A', 'Emergency international assistance for peace, normalcy and rehabilitation in Tajikistan')
 2000/... Tolerance and pluralism as indivisible elements in the promotion and protection of human r

In [164]:
# delete extracted text
split_text = re.split(draft_pattern, test, flags=re.IGNORECASE)
remain_text = split_text[1]




# Body title (H)
# split the text into paragraphs
body_title = ''
paragraphs = remain_text.split('\n\n')
body_idx = 1
body_title = ''

# for paragraph in paragraphs[1:]:
#     # check if this paragraph contains the beginning pattern of the body text
#     # lines = remain_text.split('\n')
#     # for line in lines:
#     #     if re.search(text_head_pattern, line):
#     #         break
#     print(paragraph)
#     if re.search(text_head_pattern, paragraph):
#         break
#     else:
#         body_title += paragraph
#         body_idx +=1
# if body_idx ==1:
#     body_title = 'N/A'
# else:
#     body_title = body_title.replace('\n', ' ') 
# print("Body title: ", body_title)


# # Body text (I)
# # extract the text content from the second non-empty paragraph
# body_text = ''
# footnote = ''
# footnote_idx = 1
# for paragraph in paragraphs[body_idx:]:
#     #for line in paragraph.split('\n'):
#         # exclude lines containing specific patterns
#     paragraph = paragraph.replace("\n", " ")
#     match = re.search(footnote_pattern, paragraph)
#     if match:
#         if match.group(0)[0] == '*':
#             footnote += match.group(0)
#         elif len(match.group(0)) < 90 and match.group(0)[0] == str(footnote_idx):
#             footnote += match.group(0)
#             footnote_idx += 1
#     else:
#         #if not re.search(r'^(\d{2}-\d{5}|[A-Za-z]/\d{2}/[A-Za-z]\.\d|[A-Za-z ]+Please recycle|Page \d{1})', paragraph):
#         if len(paragraph) > 50 or paragraph == "The General Assembly,":
#             body_text += paragraph.strip() + ' '
#         else:
#             if len(paragraph) == 0:
#                 continue
#             if (paragraph[0] == str(footnote_idx) and paragraph[1] == '/') or paragraph[0] == "*":
#                 footnote_idx += 1
#                 footnote += paragraph 
#             else:
#                 body_text += paragraph.strip() + ' '
# print(footnote)


N/A
Towards full integration of persons with disabilities in society:  implementation of the Standard Rules on the Equalization of  Opportunities for Persons with Disabilities, and of the long-term  strategy to implement the world programme of action concerning disabled persons to the year 2000 and beyond  


In [20]:

text = 'the quick brown fox 2004 Jumps over the Lazy Dog'
match = re.search(r'[A-Z\d]', text)

if match:
    index = match.start()
    result = text[index-1:]
    print(result)  # output: Jumps over the Lazy Dog
else:
    print('No uppercase letter or digit found.')
# use regular expression to find the index of the first uppercase letter or digit


 2004 Jumps over the Lazy Dog


In [17]:
result

'umps over the Lazy Dog'